In [244]:
# import imageio
# import numpy as np

# im = imageio.imread('leena.png')
# print(im.shape)

In [284]:
import matplotlib.pyplot as plt
import lorem
import imageio
import numpy as np
import math
from math import log10, sqrt
import cv2
import numpy as np


def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0): # MSE is zero means no noise is present in the signal .
                # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr


def toBinary(a):
    l,m=[],[]
    for i in a:
        l.append(ord(i))
    for i in l:
        m.append(int(bin(i)[2:]))
    result = []
    for x in m:
        if len(str(x)) == 8:
            result.append(str(x))
        else:
            result.append('0'*(8-len(str(x))) + str(x))
        
    return result

#reading original image 

im = imageio.imread('leena.png')
print(im.shape)

row = 512
col = 512

#determining secret to be embeded

psnr_record = []

for lsb in range(1,8):

    print("Maximum secret length in bits for LSB {} can be - {}".format(lsb, lsb*512*512*3))


    embed_secret = lorem.text()

    while (len(embed_secret)*8 < 512*512*3*lsb):

        embed_secret = embed_secret + lorem.text()

    embed_secret = embed_secret[:int(512*512*3*lsb/8)]


    # bits = "{0:08b}".format(pix)
#     print(len(embed_secret))

    print("Size of Embedding messagein bits : {}".format(len(embed_secret)*8))

    print("Converting secret to bits for embedding")

#     print(len("".join(toBinary(embed_secret))))
    message = "".join(toBinary(embed_secret))





    #emdedding secret by going to each pixel
    steg_img = []
     #count the bits embedding from the message to while embedding

    for plane in range(0,3):
        bit_counter = 0
        c = []
        for i in range(0,512):
            r = []
            for j in range(0,512):

    #             im[i,j,plane]
                bits = "{0:08b}".format(im[i,j,plane])

                bits = bits[:-lsb] + message[bit_counter*lsb:bit_counter*lsb + lsb]
    #             print(len(bits))
                bit_counter += 1
                pixel_val = int(bits,2)

    #             print("O" : {} - {}".format(im[i,j,plane], pixel_val))

                r.append(pixel_val)

            c.append(r)
        steg_img.append(c)

    #extract the embedding to match results

    # secret = ""
    # for i in range(0,int(len(message)/8)):
    #     secret = secret + (chr(int(str(message[i*8:i*8+8]),2)))

    # print(len(secret))



    #creating new steg image
    arr3 = np.dstack([steg_img[0],steg_img[1],steg_img[2]])
#     print(arr3.shape)
    arr3 = arr3.astype(np.uint8)

    plt.imsave("steg_img/" + "fs_"+str(lsb)+".png", arr3)


    #comapring PSNR


    original = cv2.imread("leena.png")
    compressed = cv2.imread("steg_img/" + "fs_"+str(lsb)+".png", 1)
    val = PSNR(original, compressed)
    print(f"PSNR value is {val} dB")
    psnr_record.append(val)


(512, 512, 3)
Maximum secret length in bits for LSB 8 can be - 6291456
Size of Embedding messagein bits : 6291456
Converting secret to bits for embedding
PSNR value is 27.935030544910894 dB


In [282]:
#plotting PSNR Values to see the drop - 

psnr_record
x1 = np.array(psnr_record)

plt.plot(x1 ,  marker = ".")

plt.ylabel("PSNR Score")
plt.xlabel("LSB Embedding Scheme")
plt.title("PSNR score of images generated after LSB embedding")
plt.grid(True)
plt.ylim(0, 100)
plt.savefig("graphs/psnr.png" , bbox_inches= 'tight')
plt.close()

In [285]:
import os
from scipy.stats import entropy
import matplotlib.pyplot as plt
import numpy as np




images = os.listdir("steg_img/")
entro = {}
for img in images:
    
    im = imageio.imread("steg_img/" + img)

    zero_bit_count = {}
    one_bit_count = {}

    #intializing the two above dict

    for i in range(0,8):
        zero_bit_count[i] = 0
        one_bit_count[i] = 0


    # counting the number of 1 and 0 in all bits
    for plane in range(0,3):
        for i in range(0,512):
            for j in range(0,512):

                bits = "{0:08b}".format(im[i,j,plane])
        #         print(bits)
                for i in range(0,len(bits)):
                    if bits[i] == "0":
                        zero_bit_count[7 - i] = zero_bit_count[7 - i] + 1

                    elif bits[i] == "1":
                        one_bit_count[7 - i] = one_bit_count[7 - i] + 1
    
    
    print("--------{}--------".format(img))
    print("Zero Bits - ")
    print(zero_bit_count)
    print("One Bits - ")
    print(one_bit_count)
    print("------------------\n")
    x1 = np.array(list(zero_bit_count.values()))
    x2 = np.array(list(one_bit_count.values()))
    
    plt.plot(x1 ,  marker = ".")
    plt.plot(x2 ,  marker = ".")
    plt.ylabel("Number of Bits")
    plt.xlabel("Position of Bit")
    plt.title("Count of Zero and One at different\n bit position for embedding {} LSB".format(img.split("_")[-1].replace(".png","")))
    plt.grid(True)
    plt.savefig("graphs/" + img, bbox_inches= 'tight')
    plt.close()
#     plt.show()
    ent = {}
# calculate entropy
    for k in zero_bit_count.keys():
        zero = zero_bit_count[k]
        one = one_bit_count[k]
        total = zero + one
        ent[k] = entropy([zero/total, one/total], base=2)
    entro[img.split("_")[-1].replace(".png","")] = ent
    

--------fs_1.png--------
Zero Bits - 
{0: 410112, 1: 403236, 2: 389406, 3: 377101, 4: 420709, 5: 433500, 6: 113878, 7: 468010}
One Bits - 
{0: 376320, 1: 383196, 2: 397026, 3: 409331, 4: 365723, 5: 352932, 6: 672554, 7: 318422}
------------------

--------fs_2.png--------
Zero Bits - 
{0: 341289, 1: 489984, 2: 389406, 3: 377101, 4: 420709, 5: 433500, 6: 113878, 7: 468010}
One Bits - 
{0: 445143, 1: 296448, 2: 397026, 3: 409331, 4: 365723, 5: 352932, 6: 672554, 7: 318422}
------------------

--------fs_3.png--------
Zero Bits - 
{0: 410154, 1: 419625, 2: 402432, 3: 377101, 4: 420709, 5: 433500, 6: 113878, 7: 468010}
One Bits - 
{0: 376278, 1: 366807, 2: 384000, 3: 409331, 4: 365723, 5: 352932, 6: 672554, 7: 318422}
------------------

--------fs_4.png--------
Zero Bits - 
{0: 461880, 1: 278061, 2: 244482, 3: 648192, 4: 420709, 5: 433500, 6: 113878, 7: 468010}
One Bits - 
{0: 324552, 1: 508371, 2: 541950, 3: 138240, 4: 365723, 5: 352932, 6: 672554, 7: 318422}
------------------

--------

In [286]:
entro

{'1': {0: 0.9986677562183462,
  1: 0.9995315484989356,
  2: 0.999932276572723,
  3: 0.9987881065532427,
  4: 0.9964707628585057,
  5: 0.992415803389092,
  6: 0.59668146004261,
  7: 0.97374179307947},
 '2': {0: 0.987383527003446,
  1: 0.9558616562817718,
  2: 0.999932276572723,
  3: 0.9987881065532427,
  4: 0.9964707628585057,
  5: 0.992415803389092,
  6: 0.59668146004261,
  7: 0.97374179307947},
 '3': {0: 0.9986611225618632,
  1: 0.9967437851289116,
  2: 0.9996037156868481,
  3: 0.9987881065532427,
  4: 0.9964707628585057,
  5: 0.992415803389092,
  6: 0.59668146004261,
  7: 0.97374179307947},
 '4': {0: 0.9778909971628794,
  1: 0.937218496931392,
  2: 0.8941808447844354,
  3: 0.6707604759685492,
  4: 0.9964707628585057,
  5: 0.992415803389092,
  6: 0.59668146004261,
  7: 0.97374179307947},
 '5': {0: 0.9961438645058233,
  1: 0.9979374904239591,
  2: 0.9919447134240733,
  3: 0.9967585753455221,
  4: 0.9998238825988729,
  5: 0.992415803389092,
  6: 0.59668146004261,
  7: 0.97374179307947},

In [287]:
 entropy([1/2,1/2], base=2)

1.0